<a href="https://colab.research.google.com/github/s97616s97616/Usual_note/blob/main/TPS_july.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline
import random
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
from sklearn.model_selection import KFold
from sklearn import preprocessing
!pip install optuna
import optuna
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
api_token = {"username":"watsons","key":"dc7da47ca9aa5e696b65f97c16fd627b"}
import json
import zipfile
import os
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')
!kaggle competitions download -c tabular-playground-series-jul-2021 --force

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
y = np.log1p(train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']])
train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1, inplace=True)

In [ ]:
df = pd.concat([train, test], ignore_index=True)

In [ ]:
df

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2
...,...,...,...,...,...,...,...,...,...
9353,2011-04-04 10:00:00,23.2,28.7,0.7568,1340.3,1023.9,522.8,1374.0,1659.8
9354,2011-04-04 11:00:00,24.5,22.5,0.7119,1232.8,955.1,616.1,1226.1,1269.0
9355,2011-04-04 12:00:00,26.6,19.0,0.6406,1187.7,1052.4,572.8,1253.4,1081.1
9356,2011-04-04 13:00:00,29.1,12.7,0.5139,1053.2,1009.0,702.0,1009.8,808.5


In [ ]:
df.drop(['date_time'], axis=1).corr()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
deg_C,1.000000,-0.579770,0.557004,0.045155,0.168660,-0.036380,0.439112,-0.043440
relative_humidity,-0.579770,1.000000,0.217142,0.110323,-0.014681,-0.147934,0.044987,0.145467
absolute_humidity,0.557004,0.217142,1.000000,0.117803,0.294134,-0.390279,0.678408,0.118908
sensor_1,0.045155,0.110323,0.117803,1.000000,0.796913,-0.587970,0.583242,0.861892
sensor_2,0.168660,-0.014681,0.294134,0.796913,1.000000,-0.802304,0.792547,0.848273
sensor_3,-0.036380,-0.147934,-0.390279,-0.587970,-0.802304,1.000000,-0.637138,-0.702862
sensor_4,0.439112,0.044987,0.678408,0.583242,0.792547,-0.637138,1.000000,0.577118
sensor_5,-0.043440,0.145467,0.118908,0.861892,0.848273,-0.702862,0.577118,1.000000


In [ ]:
def add_new_plot_features(df):
  df["month"] = df["date_time"].dt.month
  df["day_of_week"] = df["date_time"].dt.dayofweek
  df["day_of_year"] = df["date_time"].dt.dayofyear
  df["hour"] = df["date_time"].dt.hour
  df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype("int")
  df["working_hours"] = df["hour"].isin(np.arange(8, 18, 1)).astype("int")
  df["is_weekend"] = (df["date_time"].dt.dayofweek >= 5).astype("int")
  return df

In [ ]:
df["date_time"] = pd.to_datetime(df["date_time"], format="%Y-%m-%d %H:%M:%S")
col = list(df.columns[1:])
df = add_new_plot_features(df)
df.drop(['date_time'], axis=1, inplace=True)

In [ ]:
col2 = df.columns
df[col] = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df[col]))
df.columns = col2

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9358 entries, 0 to 9357
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   deg_C              9358 non-null   float64
 1   relative_humidity  9358 non-null   float64
 2   absolute_humidity  9358 non-null   float64
 3   sensor_1           9358 non-null   float64
 4   sensor_2           9358 non-null   float64
 5   sensor_3           9358 non-null   float64
 6   sensor_4           9358 non-null   float64
 7   sensor_5           9358 non-null   float64
 8   month              9358 non-null   int64  
 9   day_of_week        9358 non-null   int64  
 10  day_of_year        9358 non-null   int64  
 11  hour               9358 non-null   int64  
 12  week_of_year       9358 non-null   int64  
 13  working_hours      9358 non-null   int64  
 14  is_weekend         9358 non-null   int64  
dtypes: float64(8), int64(7)
memory usage: 1.1 MB


In [ ]:
train = df.iloc[:7111,:]
test = df.iloc[7111:,:]

In [ ]:
lgbr = lgb.LGBMRegressor(max_depth=5,
            n_estimators=1000,
            subsample=0.8,
            colsample_bytree=0.9,
            reg_alpha=1,
            reg_lambda=1
            )
kf = KFold(n_splits=5, random_state=23, shuffle=True)
for col in range(len(y.columns)):
  x_train, x_test, y_train, y_test = train_test_split(train, y.iloc[:,col], test_size=0.2, random_state=42)
  lgbr.fit(x_train, y_train)
  y_pred = lgbr.predict(x_train)
  from sklearn.metrics import mean_squared_error
  print(round(np.sqrt(mean_squared_error(y_pred, y_train)),3))
  scores = cross_val_score(lgbr, train, y.iloc[:,col], scoring='neg_root_mean_squared_error', cv=kf)
  score = -round(scores.mean(),3)
  print(score)
  print('---------------------------------')

0.063
0.095
---------------------------------
0.055
0.083
---------------------------------
0.091
0.195
---------------------------------


In [ ]:
out=pd.DataFrame()
for i in range(3):
  lgbm = lgb.LGBMRegressor(max_depth=5,
            n_estimators=1000,
            subsample=0.8,
            colsample_bytree=0.9,
            reg_alpha=1,
            reg_lambda=1)
  lgbm.fit(train, y.iloc[:,i])
  tmp = pd.DataFrame(lgbm.predict(test))
  out = pd.concat([out,tmp.iloc[:,0]],axis=1)

In [ ]:
test

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,month,day_of_week,day_of_year,hour,week_of_year,working_hours,is_weekend
7111,0.204593,0.395604,0.123540,0.332766,0.200113,0.215571,0.149180,0.443766,1,5,1,0,52,0,1
7112,0.144050,0.522589,0.132776,0.428610,0.261354,0.167184,0.188002,0.629346,1,5,1,1,52,0,1
7113,0.158664,0.520147,0.138885,0.328542,0.268085,0.169754,0.175075,0.455426,1,5,1,2,52,0,1
7114,0.141962,0.529915,0.139080,0.353815,0.287711,0.183889,0.203983,0.447933,1,5,1,3,52,0,1
7115,0.131524,0.593407,0.136979,0.273910,0.247791,0.248538,0.185576,0.388711,1,5,1,4,52,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,0.521921,0.241758,0.279578,0.490463,0.343044,0.094027,0.355840,0.606533,4,0,94,10,14,1,0
9354,0.549061,0.166056,0.257636,0.417234,0.307696,0.135369,0.293968,0.442041,4,0,94,11,14,1,0
9355,0.592902,0.123321,0.222792,0.386512,0.357686,0.116182,0.305388,0.362951,4,0,94,12,14,1,0
9356,0.645094,0.046398,0.160876,0.294891,0.335388,0.173431,0.203481,0.248211,4,0,94,13,14,1,0


In [ ]:
for i in range(3):
  sample_submission.iloc[:,i+1] = np.expm1(out.iloc[:,i])
sample_submission

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,1.434885,4.811909,178.800352
1,2011-01-01 01:00:00,1.460301,7.026328,167.349932
2,2011-01-01 02:00:00,1.495643,6.861748,156.844956
3,2011-01-01 03:00:00,1.642711,7.664061,181.987125
4,2011-01-01 04:00:00,0.986894,5.713795,104.362567
...,...,...,...,...
2242,2011-04-04 10:00:00,2.532256,13.272444,217.585306
2243,2011-04-04 11:00:00,2.389148,11.719016,176.987880
2244,2011-04-04 12:00:00,2.513401,12.833483,168.953466
2245,2011-04-04 13:00:00,2.010444,11.192270,127.201134


In [ ]:
y.describe()

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
count,7111.000000,7111.000000,7111.000000
mean,1.031356,2.168513,4.949659
std,0.429461,0.761459,0.881815
min,0.095310,0.095310,1.064711
25%,0.693147,1.704748,4.349632
50%,0.993252,2.251292,4.955827
75%,1.335001,2.721295,5.564520
max,2.602690,4.169761,7.295260


In [ ]:
sample_submission.to_csv('upload.csv',index=False)
!kaggle competitions submit -c tabular-playground-series-jul-2021 -f upload.csv -m "upload"

100% 166k/166k [00:11<00:00, 14.9kB/s]
Successfully submitted to Tabular Playground Series - Jul 2021